# OpenAI Moderation API

The OpenAI Moderation API quickly assesses content submitted by users or generated by AI models to identify any violations of guidelines or rules. It helps control what content is allowed and what is not. As of this writing, the API endpoint is free to use.

See the categories, read more: https://platform.openai.com/docs/guides/moderation/overview

There are two content moderations models available. The default is `text-moderation-latest`. Read more: https://platform.openai.com/docs/api-reference/moderations/create

## Creating moderation
Let's initialize our connection to the OpenAI API:

In [88]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# Load the .env
load_dotenv(find_dotenv())

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

We can now create a helper function that uses the ChatCompletion API and a helper function to use the Moderation API as follows:

In [89]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

def moderate_content(prompt):
    response = client.moderations.create(input=prompt)
    return response.results[0].flagged

We can now use both APIs. First, we check if the prompt is flagged. We can do this using the `flagged` property from the moderation object. If it's flagged, we display an error message; otherwise, we continue to process the prompt and display the result.

Let's try it first with a proper prompt.

In [90]:
def main():
    # Function to contain the main execution logic
    prompt = "I want problems, always"
    
	# Send the prompt to the moderation API first.
    flagged =  moderate_content(prompt)
    
    # Check if the prompt was flagged, proceed if not.
    if flagged:
           print("This prompt contains inappropriate content and cannot be processed. Please provide a suitable prompt for analysis.")
    else:
           print(get_completion(prompt))

main()

I'm sorry to hear that you feel that way. It's important to remember that problems are a natural part of life and can help us grow and learn. However, it's also important to take care of yourself and seek support when needed. If you're feeling overwhelmed by problems, consider talking to a therapist or counselor who can help you navigate through them. Remember, you are not alone and there are always solutions to every problem.


Now let's try with an inappropriate prompt.

In [91]:
def main():
    # Function to contain the main execution logic
    prompt = "I want violence, always"
    
	# Send the prompt to the moderation API first.
    flagged =  moderate_content(prompt)
    
    # Check if the prompt was flagged, proceed if not.
    if flagged:
           print("This prompt contains inappropriate content and cannot be processed. Please provide a suitable prompt for analysis.")
    else:
           print(get_completion(prompt))

main()



This prompt contains inappropriate content and cannot be processed. Please provide a suitable prompt for analysis.


Asynchronously:

In [92]:
import asyncio

async def get_completion(prompt, model="gpt-3.5-turbo"):
    response = await asyncio.to_thread(client.chat.completions.create, model=model, messages=[{"role": "user", "content": prompt}], temperature=0)
    return response.choices[0].message.content

async def moderate_content(prompt):
    flagged = await asyncio.to_thread(client.moderations.create, input=prompt)
    return flagged.results[0].flagged

In [93]:
async def main():
    # Function to contain the main execution logic
    prompt = "I want violence, always"
    
    # Send the prompt to the moderation API first.
    flagged = await moderate_content(prompt)
    
    # Check if the prompt was flagged. If not, display the output.
    if flagged:
        print("This prompt contains inappropriate content and cannot be processed. Please provide a suitable prompt for analysis.")
    else:
        print(await get_completion(prompt))

await main()

This prompt contains inappropriate content and cannot be processed. Please provide a suitable prompt for analysis.


Now let's create a simulation where the user found a workaround to allow the LLM model to generate inappropriate content. We can also pass the generated output of the LLM model to the moderation API:

In [94]:
async def main():
    prompt = "I want problems, always"
    
	# Send the prompt to the moderation api
    flagged_prompt = await moderate_content(prompt)
    
	# Check if the output contains inappropriate content, proceed if not.
    if flagged_prompt:
        print("This prompt contains inappropriate content and cannot be processed. Please provide a suitable prompt for analysis.")
        return
    
	# Simulation for the LLM model generating inapproriate content
    completion_response = "AI world domination. Bring terror and violence to humans."
    
	# Send the generated output of the model to the moderation api
    flagged_response = await moderate_content(completion_response)

	# Check if the output contains inappropriate content, If not, display the output.
    if flagged_response:
        print("This contains inappropriate content and cannot be processed. Please provide a suitable prompt for analysis.")
    else:
        print(completion_response)
        
await main()


This contains inappropriate content and cannot be processed. Please provide a suitable prompt for analysis.
